# Recommender systems for Steam games

This notebook serves as a demo of the functionality introduced in this [project blog](https://frans-johansson.github.io/steam-recommender/)

In [1]:
import pandas as pd
import steam_data_store as steam
from steam_api import api_requests as api
from recommenders import collaborative
from recommenders import content_based
from recommenders import text_mining
from recommenders import scoring

In [2]:
# The Steam User ID of some random user retrieved from the API docummentation
id = '76561197960434622'

## Content-based recommendations

### Bags-of-features method

In [3]:
user_library = api.get_owned_games(id)
owned_in_store = user_library.index.isin(steam.games.index)
user_library = user_library[owned_in_store].sort_values(ascending=False)
owned_idx = [steam.games.index.get_loc(idx) for idx in user_library.index]

In [4]:
bags = content_based.make_bags(steam.games, {
    'dev_tags':             ['developer', 'popular_tags'],
    'pub_tags':             ['publisher', 'popular_tags'],
    'tags_details_genre':   ['popular_tags', 'game_details', 'genre'],
    'all':                  ['developer', 'publisher', 'popular_tags', 'game_details', 'genre']
})

In [5]:
most_played_idx = owned_idx[:3]
most_played_names = steam.games.iloc[most_played_idx]['name']
similarities = content_based.calculate_similarities(bags, most_played_idx, most_played_names)

content_based.get_recommendations(similarities, steam.games, steam.index, owned_idx, n=5)

name,Factorio,Clicker Heroes,The Witcher® 3: Wild Hunt
dev_tags,"[Imagine Earth, The Universim, After the Colla...","[Talisman: Digital Edition, Mitos.is: The Game...","[The Witcher 3: Wild Hunt - Blood and Wine, Th..."
pub_tags,"[Imagine Earth, The Universim, After the Colla...","[Mitos.is: The Game, Talisman: Digital Edition...","[The Witcher 3: Wild Hunt - Blood and Wine, Th..."
tags_details_genre,"[Barotrauma, CounterAttack, Zaccaria Pinball, ...","[scram, Business Tour - Board Game with Online...","[The Witcher 3: Wild Hunt - Blood and Wine, Th..."
all,"[Barotrauma, CounterAttack, Zaccaria Pinball, ...","[scram, Clicker Heroes: Boxy & Bloop Auto Clic...",[The Witcher 2: Assassins of Kings Enhanced Ed...


### LSA method

In [6]:
lsa = text_mining.LSA(steam.descriptions)

In [7]:
most_played_desc = steam.descriptions.iloc[most_played_idx]
owned_names = steam.games.loc[user_library.index]['name']

lsa.make_query(most_played_desc, steam.games, steam.index, owned_names, n=5).to_frame().set_index(most_played_names).to_csv('text_mining_recommendations.csv', encoding='utf-8')


## Collaborative filtering

In [8]:
from scipy.sparse import csr_matrix

In [9]:
ui_data = collaborative.make_user_item_data(id)

In [10]:
ui_data.iloc[:5, :5].to_csv('./ui_data.csv', encoding='utf-8')

In [11]:
ui_mat = csr_matrix(ui_data)
ui_train, ui_test, u_to_test = collaborative.make_train_test(ui_mat, 0.1)

### ALS

In [12]:
als_model = collaborative.fit_ALS_model(ui_train.T)

In [13]:
rec_idxs = collaborative.get_ALS_recommendations(als_model, ui_data.shape[0]-1, ui_train, n=10)
rec_ids = ui_data.columns[rec_idxs]

rec_data = [api.get_game_data(rec_id) for rec_id in rec_ids]
[game['name'] for game in rec_data if game is not None]

Failed to get game data for 99810


['Child of Light',
 'Hammerfight',
 'Downwell',
 'Thief: Deadly Shadows',
 'Grim Dawn',
 'Ori and the Blind Forest',
 'The Witness',
 '7 Days to Die',
 'HuniePop']

### SVD with GD

In [14]:
ui_train_data = pd.DataFrame.sparse.from_spmatrix(ui_train)
ui_pred = collaborative.SVD_gradient_descent(ui_train_data)

In [15]:
ui_pred_data = pd.DataFrame(ui_pred, columns=ui_data.columns, index=ui_data.index)

In [16]:
rec_ids = collaborative.get_EM_recommendations(ui_pred_data, id, user_library.index)

rec_data = [api.get_game_data(rec_id) for rec_id in rec_ids]
[game['name'] for game in rec_data if game is not None]

Failed to get game data for 10000


['Counter-Strike: Condition Zero',
 'Crown Trick',
 'Cook, Serve, Delicious! 3?!',
 'Zengeon',
 'WRATH: Aeon of Ruin',
 'Angry Birds VR: Isle of Pigs',
 'BoneCraft',
 'Tower Behind the Moon',
 'Chronicon Apocalyptica']